# Agenda
  - Train the sklearn model
  - save in pkl/ jobib format
  - load pkl model (check size, inference speed)
  - convert model format to onnx
  - save onnx model (.onnx) format
  - load onnx model and inference (observe file size, and inference time)

## Documentation
https://onnxruntime.ai/docs/api/python/tutorial.html

https://onnx.ai/sklearn-onnx/api_summary.html#skl2onnx.convert_sklearn

https://onnx.ai/sklearn-onnx/auto_tutorial/plot_abegin_convert_pipeline.html

In [21]:
import os
os.chdir("/content/drive/MyDrive/onnx_runtime/machine learning")

In [22]:
!pwd

/content/drive/MyDrive/onnx_runtime/machine learning


In [5]:
# !touch requirements.txt
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.3/300.3 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.4
    Uninstalling protobuf-5.29.4:
      Successfully uninstalled protobuf-5.29.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.16.1 requires protobuf<6.0.0dev,>=4.25.2; python_version >= "3.11", but you have proto

In [1]:
!pip list

Package                            Version
---------------------------------- -------------------
absl-py                            1.4.0
accelerate                         1.5.2
aiohappyeyeballs                   2.6.1
aiohttp                            3.11.14
aiosignal                          1.3.2
alabaster                          1.0.0
albucore                           0.0.23
albumentations                     2.0.5
ale-py                             0.10.2
altair                             5.5.0
annotated-types                    0.7.0
anyio                              4.9.0
argon2-cffi                        23.1.0
argon2-cffi-bindings               21.2.0
array_record                       0.7.1
arviz                              0.21.0
astropy                            7.0.1
astropy-iers-data                  0.2025.3.24.0.35.32
astunparse                         1.6.3
atpublic                           5.1
attrs                              25.3.0
audioread            

# Train the traditional sklearn model

In [52]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib
import pandas as pd

In [53]:
iris= load_iris()
X,y= iris.data, iris.target
X_train, X_test, y_train, y_test= train_test_split(X,y,test_size=0.2,random_state=42)

In [54]:
!ls

joblib_output  machine_learning_demo.ipynb  onnx_model	requirements.txt


In [56]:
clf= RandomForestClassifier().fit(X_train,y_train)
joblib.dump(clf,"joblib_output/clf.pkl")

['joblib_output/clf.pkl']

In [79]:
%time
preds_jolib= clf.predict(X_test)
preds_jolib

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 14.1 µs


array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0])

In [58]:
clf

RandomForestClassifier()

In [59]:
# check the size of model
!ls -lh joblib_output/clf.pkl

-rw------- 1 root root 183K Apr  2 09:55 joblib_output/clf.pkl


In [60]:
import os

file_path = "joblib_output/clf.pkl"  # Replace with your file path
file_size_bytes = os.path.getsize(file_path)
file_size_mb = file_size_bytes / (1024 * 1024)  # Convert bytes to megabytes
print(f"File size Bytes: {file_size_bytes} bytes")
print(f"File size MB: {file_size_mb:.2f} MB")

File size Bytes: 186401 bytes
File size MB: 0.18 MB


In [61]:
y_train[:4]

array([0, 0, 1, 0])

# Convert to onnx runtime

https://onnx.ai/sklearn-onnx/api_summary.html

In [62]:
from skl2onnx import convert_sklearn # convert skl-> onnx
from skl2onnx.common.data_types import FloatTensorType

In [64]:
# load joblib model
cls= joblib.load("joblib_output/clf.pkl")
cls

RandomForestClassifier()

In [65]:
initial_type = [('float_input', FloatTensorType([None, 4]))]
onx = convert_sklearn(cls, initial_types=initial_type)
with open("onnx_model/random_forest.onnx", "wb") as f:
    f.write(onx.SerializeToString())

In [67]:
type(onx)

onnx.onnx_ml_pb2.ModelProto

In [37]:
print(onx)

ir_version: 10
producer_name: "skl2onnx"
producer_version: "1.18.0"
domain: "ai.onnx"
model_version: 0
doc_string: ""
graph {
  node {
    input: "float_input"
    output: "label"
    output: "probabilities"
    name: "TreeEnsembleClassifier"
    op_type: "TreeEnsembleClassifier"
    attribute {
      name: "class_ids"
      ints: 0
      ints: 1
      ints: 2
      ints: 0
      ints: 1
      ints: 2
      ints: 0
      ints: 1
      ints: 2
      ints: 0
      ints: 1
      ints: 2
      ints: 0
      ints: 1
      ints: 2
      ints: 0
      ints: 1
      ints: 2
      ints: 0
      ints: 1
      ints: 2
      ints: 0
      ints: 1
      ints: 2
      ints: 0
      ints: 1
      ints: 2
      ints: 0
      ints: 1
      ints: 2
      ints: 0
      ints: 1
      ints: 2
      ints: 0
      ints: 1
      ints: 2
      ints: 0
      ints: 1
      ints: 2
      ints: 0
      ints: 1
      ints: 2
      ints: 0
      ints: 1
      ints: 2
      ints: 0
      ints: 1
      ints: 2
      i

In [68]:
## Size of onnx model file
import os

file_path = "onnx_model/random_forest.onnx"  # Replace with your file path
file_size_bytes = os.path.getsize(file_path)
file_size_mb = file_size_bytes / (1024 * 1024)  # Convert bytes to megabytes
print(f"ONXX File size Bytes: {file_size_bytes} bytes")
print(f"ONXX File size MB: {file_size_mb:.2f} MB")

ONXX File size Bytes: 79939 bytes
ONXX File size MB: 0.08 MB


In [92]:
(0.18 - 0.08)

0.09999999999999999

## -- model size reduced to 0.08MB from 0.18 Mb

## Load and run Onnx Model

In [77]:
import onnxruntime as rt
import numpy as np

sess = rt.InferenceSession(
    "onnx_model/random_forest.onnx",
    providers=rt.get_available_providers())

input_name= sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
pred_onx = sess.run(
    [label_name], {input_name: X_test.astype(np.float32)})[0]

In [93]:
rt.get_available_providers()

['AzureExecutionProvider', 'CPUExecutionProvider']

In [87]:
%time
pred_onx = sess.run(
    [label_name], {input_name: X_test.astype(np.float32)})[0]

pred_onx

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.87 µs


array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0], dtype=int64)

## - Now you can see the, inference time also reduced to 3microsec from 6

# Visualize using netron
https://netron.app/

In [38]:
# !pip install netron -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 16.1 MB/s eta 0:00:00


In [45]:
# prompt: how to visualize the onnx model

# import netron

# # Replace with the actual path to your ONNX model
# onnx_model_path = "onnx_model/random_forest.onnx"

# # Visualize the ONNX model using Netron
# netron.start(onnx_model_path)
